In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 9.2 MB/s 


In [3]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=708bbb33999cc633d434f6a4b1abfa11822b2f587ac5c94b5b364b57ee3bd10b
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [13]:
!pip install fracdiff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 10.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
from sklearn.preprocessing import StandardScaler
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings(action='ignore')
from matplotlib import font_manager, rc
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


from fracdiff import *
# font_path = "C:\\Windows\\Fonts\\\x7f\x7f\x7f\x7fBOLD.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
from sklearn.manifold import TSNE
import re
import talib as tb

In [15]:
def weather(주소,name = 'train'):
    t=0
    if name =='test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):
        globals()[f'{name}_weather_'+str(i)]=pd.DataFrame()

        for k in range(0,3):
            try:
                weather = pd.read_csv(주소+f'/weather_{i}_{t}.csv')
                if len(weather)==0:
                    weather = pd.read_csv(주소+f'/weather_0_0.csv')
                    weather=weather.set_index('datadate')
                    weather[weather.columns] = 0
                    weather=weather.reset_index()
                weather = weather.set_index('datadate')
                weather['습도(%)']=weather['습도(%)'].apply(lambda x : np.nan if x==' ' else x)
                weather['습도(%)']=weather['습도(%)'].astype('float')
                weather.columns = weather.columns+f'_{k%3}'
                globals()[f'{name}_weather_'+str(i)]=pd.merge(globals()[f'{name}_weather_'+str(i)],weather,how='right',left_index=True, right_index=True)    


            except:
                break
            t+=1



        globals()[f'{name}_weather_'+str(i)]=globals()[f'{name}_weather_'+str(i)].reset_index()
        cols = globals()[f'{name}_weather_'+str(i)].columns
        globals()[f'{name}_weather_'+str(i)]['월일']=globals()[f'{name}_weather_'+str(i)]['datadate']%10000
       

        
        if name == 'train':
            qwe =  globals()[f'{name}_weather_'+str(i)].groupby('월일').mean().reset_index()
            qwe.columns = qwe.columns+'_평균'
            globals()[f'train_weather_평균_'+str(i)]= qwe



            


        globals()[f'{name}_weather_'+str(i)] = pd.merge(globals()[f'{name}_weather_'+str(i)],globals()[f'train_weather_평균_'+str(i)],how='left',left_on = '월일',right_on='월일_평균')
        for c in cols:
            globals()[f'{name}_weather_'+str(i)][c]=globals()[f'{name}_weather_'+str(i)].apply(lambda x: x[c] if np.isnan(x[c]) == False else (0 if np.isnan(x[c+'_평균']) else x[c+'_평균']),axis=1)
                
    
        
        
        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_weather_'+str(i)] = globals()[f'{name}_weather_'+str(i)][cols]

        else:
            globals()[f'{name}_weather_'+str(i)] = globals()[f'{name}_weather_'+str(i)][cols]

        

In [16]:
def add_dosomae(주소,name='train', option=1):
        if name == 'test':
            test_dir = 주소.split('/')[-2][-1]
        data_list = glob(주소+'*') # train raw 데이터 넣을지 / test raw 데이터 넣을지 경로
        domae = []
        somae = []

        for i in data_list:
            if 'domae' in i:
                domae.append(i)
            if 'somae' in i:
                somae.append(i)
        
        sum_df = pd.DataFrame()
                
        if option == 1:
            df = domae
            text = 'domae'
        else:
            df = somae
            text = 'somae'


        for i in tqdm(df):
            test = pd.read_csv(i)
            
            k=i.split('/')[-1].split("_")[1].split(".")[0]
            test.fillna(0,inplace=True) # 널값 0으로 채워주고
            if len(test)==0:
                test = pd.read_csv(주소+f'/domae_0.csv')
                test=test.set_index('datadate')
                test[test.columns] = 0
                test=test.reset_index()
                

            
            test['조사단위(kg)'] = test['조사단위(kg)'].replace('g$|kg$|개|포기|접', '',regex=True)
            test=test.astype({'조사단위(kg)':'float'})
            test.loc[test['조사단위(kg)'] >= 100, '단위당가격'] = test['가격(원)']* test['조사단위(kg)'] /1000
            test.loc[test['조사단위(kg)'] < 100, '단위당가격'] = test['가격(원)']/ test['조사단위(kg)']
            
            sep = test.loc[(test['등급명'] == '상품') | (test['등급명'] == 'S과') | (test['등급명'] == 0)]   # 모든 상품에 대해서 수행하지 않고 GRAD_NM이 '상품', 'S과' 만 해당하는 품목 가져옴
            sep = sep[['datadate', '등급명', '조사단위(kg)', '가격(원)']]

            sep.rename(columns={"가격(원)": "가격"}, inplace=True)
            sep2 = sqldf(
                f"select datadate, max(가격) as '일자별_{text}가격_최대(원)', avg(가격) as '일자별_{text}가격_평균(원)', min(가격) as '일자별_{text}가격_최소(원)' from sep group by datadate")

            sep2.fillna(0,inplace=True)
#             if len(sep2) == 0:
#                 if name=='test':
#                     sep2 =  globals()['sep_'+test_dir+f'_{name}_{text}_0'] 
#                     sep2

#                 else:
#                     sep2 =  globals()[f'{name}_{text}_0'] 
#                     sep2[sep2.columns] = 0 
                
                
            if name=='test':
                globals()['sep_'+test_dir+f'_{name}_{text}_{k}'] = sep2
                
            else:
                globals()[f'{name}_{text}_{k}'] = sep2
 
                


        
                               
               
             

In [17]:
def pummok(주소,name='train'):
    if name == 'test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):

        try:
            pummok = pd.read_csv(주소+f'/pummok_{i}.csv')
            if len(pummok) == 0 :
                pummok = pd.read_csv(주소+f'/pummok_0.csv')
                pummok = pummok.groupby('datadate').mean()
                pummok = pummok[['단가(원)','거래량','해당일자_전체평균가격(원)']]
                pummok[pummok.columns] = 0
                pummok = pummok.reset_index()
                
            else:   
                pummok = pummok.groupby('datadate').mean()
                pummok = pummok[['단가(원)','거래량','해당일자_전체평균가격(원)']]

                # 새로 추가한 부분
                pummok['해당일자_전체평균가격(원)'].fillna(0, inplace=True)
                pummok['wma14'] =  tb.WMA(pummok['해당일자_전체평균가격(원)'], 14)
                pummok['wma14'].fillna(method='bfill', inplace=True)

                # wma * 1.2 적용  -> 그림이 제일 비슷하게 나온다.
                pummok['해당일자_전체평균가격(원)'] = np.where(pummok['해당일자_전체평균가격(원)']==0, pummok['wma14']*1.2, pummok['해당일자_전체평균가격(원)'])
                pummok.drop('wma14', axis=1, inplace=True)


                
                pummok = pummok.reset_index()
                pummok.fillna(0, inplace = True)
                # pummok.fillna(0,inplace = True)
                
                
                

        except:
            continue

        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_pummok_{i}'] = pummok

        else:
            globals()[f'{name}_pummok_{i}'] = pummok


In [18]:
def imexport(주소, name='train'):
    if name == 'test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):
        try:
            imexport=pd.read_csv(주소+f'/imexport_{i}.csv')
            
            if len(imexport) == 0:
                imexport = pd.read_csv(주소+f'/imexport_0.csv')
                imexport = imexport.groupby('datadate').mean()
                imexport[imexport.columns] = 0

            else:  
                imexport = imexport.groupby('datadate').mean()


        except:
            
            imexport = pd.read_csv(주소+f'/imexport_0.csv')
            imexport = imexport.groupby('datadate').mean()
            imexport[imexport.columns] = 0
            pass


        imexport.fillna(0,inplace=True)
        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_imexport_{i}'] = imexport

        else:
            globals()[f'{name}_imexport_{i}'] = imexport


    
    

In [19]:
def make_csv(주소,name = 'train'):
    imexport(주소,name)
    pummok(주소,name)
    add_dosomae(주소,name)
    add_dosomae(주소,name,2)
    weather(주소,name)
    for i in range(0,37):
        if name == 'test':

            num = 주소.split('/')[-2][-1]
            
            
            temp = globals()[f'sep_{num}_{name}_pummok_{i}']

            temp1 = globals()[f'sep_{num}_{name}_weather_{i}']

            temp2 = globals()[f'sep_{num}_{name}_domae_{i}']
   
            temp3 = globals()[f'sep_{num}_{name}_somae_{i}']

            temp4 = globals()[f'sep_{num}_{name}_imexport_{i}']
            temp.set_index('datadate',inplace=True)
            temp1.set_index('datadate',inplace=True)
            temp2.set_index('datadate',inplace=True)
            temp3.set_index('datadate',inplace=True)

            temp = temp.join(temp1)
            temp = temp.join(temp2)
            temp = temp.join(temp3)
            temp.reset_index(inplace = True)
            temp['datadate'] = temp['datadate'].astype('str')
            temp['yyyymm'] =  temp['datadate'].apply(lambda x: x[:6] )
            temp4.reset_index(inplace=True)
            temp4['yyyymm'] = temp4['datadate'].astype('str')
            temp4.drop('datadate',inplace=True,axis=1)
            temp = temp.merge(temp4, how='left',on='yyyymm')
            temp.set_index('datadate',inplace = True)
            temp.drop('yyyymm',inplace=True,axis=1)
            a = fdiff(temp['해당일자_전체평균가격(원)'], n=0.35)
            diff = pd.DataFrame(a, index=temp.index, columns=["fracdiff"])
            temp['fracdiff'] = diff['fracdiff']


            temp.fillna(0, inplace = True) # 내가 추가한 코드
            globals()[f'{name}_total_sep_{num}_{i}'] = temp


        else:
            temp = globals()[f'{name}_pummok_{i}']
            temp1 = globals()[f'{name}_weather_{i}']
            temp2 = globals()[f'{name}_domae_{i}']
            temp3 = globals()[f'{name}_somae_{i}']
            temp4 = globals()[f'{name}_imexport_{i}']
            temp.set_index('datadate',inplace=True)
            temp1.set_index('datadate',inplace=True)
            temp2.set_index('datadate',inplace=True)
            temp3.set_index('datadate',inplace=True)
            temp = temp.join(temp1)
            temp = temp.join(temp2)
            temp = temp.join(temp3)
            temp.reset_index(inplace = True)
            temp['datadate'] = temp['datadate'].astype('str')
            temp['yyyymm'] =  temp['datadate'].apply(lambda x: x[:6] )
            temp4.reset_index(inplace=True)
            temp4['yyyymm'] = temp4['datadate'].astype('str')
            temp4.drop('datadate',inplace=True,axis=1)
            temp = temp.merge(temp4, how='left',on='yyyymm')
            temp.set_index('datadate',inplace = True)
            temp.drop('yyyymm',inplace=True,axis=1)
            a = fdiff(temp['해당일자_전체평균가격(원)'], n=0.35)
            diff = pd.DataFrame(a, index=temp.index, columns=["fracdiff"])
            temp['fracdiff'] = diff['fracdiff']

            temp.fillna(0, inplace = True)
            globals()[f'{name}_total_{i}'] = temp
        
    

In [20]:
# train_data만들기 train_total_0 ~ train_total_36까지 만들어준다.
make_csv('/content/drive/MyDrive/농산물예측/aT_data/aT_train_raw/','train')

100%|██████████| 37/37 [00:25<00:00,  1.43it/s]


In [21]:
def make_smoothing(df):
  df['해당일자_전체평균가격(원)'] = tb.WMA(df['해당일자_전체평균가격(원)'], 7)
  df.fillna(method='bfill', inplace=True)
  return df

In [22]:
# train_data smoothing 해주기~
for i in range(37):
  globals()[f'train_smoothing_{i}'] = make_smoothing(globals()[f'train_total_{i}'])

In [23]:
# nan이 생겼나 확인
for i in range(37):
  print(len(globals()[f'train_smoothing_{i}']), globals()[f'train_smoothing_{i}'].isna().sum().sum())

1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0
1461 0


In [24]:
# OLS Feature selection을 위한 함수
def OLS_report(df, seed):
    
    
    
    if len(df.columns)<len(train_smoothing_0.columns): # 주산지2의 날씨 데이터가 없다. 예외처리
      model = sm.OLS.from_formula("해당일자_전체평균가격 ~단가+거래량+초기온도_0+최대온도_0+최저온도_0+ 평균온도_0+강수량_0+ 습도_0+초기온도_1+최대온도_1+최저온도_1+ 평균온도_1+강수량_1+ 습도_1+일자별_domae가격_최대+일자별_domae가격_평균+ 일자별_domae가격_최소+ 일자별_somae가격_최대+일자별_somae가격_평균+ 일자별_somae가격_최소+ 수출중량+ 수출금액+ 수입중량+ 수입금액+무역수지"
                                ,data=df)
    else:
      model = sm.OLS.from_formula("해당일자_전체평균가격 ~단가+거래량+초기온도_0+최대온도_0+최저온도_0+ 평균온도_0+강수량_0+ 습도_0+초기온도_1+최대온도_1+최저온도_1+ 평균온도_1+강수량_1+ 습도_1+초기온도_2+최대온도_2+최저온도_2+ 평균온도_2+강수량_2+ 습도_2 +일자별_domae가격_최대+일자별_domae가격_평균+ 일자별_domae가격_최소+ 일자별_somae가격_최대+일자별_somae가격_평균+ 일자별_somae가격_최소+ 수출중량+ 수출금액+ 수입중량+ 수입금액+무역수지"
                                ,data=df)
    return model.fit()




def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    # conf_lower = results.conf_int()[0]
    # conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff
                              #  "conf_lower":conf_lower,
                              #  "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals"]]
    return results_df





def OLS_feature_selection(df):
  df.columns = df.columns.str.replace(r'\([^)]*\)','',regex= True) # 컬럼에서 문자 기호 같은거 있으면 OLS 안돌아간다.     
                                                                  # 테스트 데이터에도 적용해야한다. 컬럼명 바꿔야하니깐.
  result = OLS_report(df,42)
  ols_report_df = results_summary_to_dataframe(result).reset_index()
  ols_report_df = ols_report_df[ols_report_df['index'] !='Intercept']
  selected_col = list(ols_report_df[ols_report_df['pvals']<0.05]['index'].values) 
  selected_df = df[selected_col]
  selected_df['해당일자_전체평균가격'] = df['해당일자_전체평균가격']
  return selected_df        

In [25]:
# 유의수준을 토대로 columns selection
for i in range(37):
  globals()[f'total_{i}'] = OLS_feature_selection(globals()[f'train_smoothing_{i}'])

In [26]:
# 품목별로 컬럼 확인
for i in range(37):
  print(f"{i} 품목의 selected_columns : {list(globals()[f'total_{i}'].columns)}")

0 품목의 selected_columns : ['단가', '습도_0', '최저온도_1', '평균온도_1', '최대온도_2', '습도_2', '일자별_somae가격_최대', '일자별_somae가격_최소', '해당일자_전체평균가격']
1 품목의 selected_columns : ['단가', '거래량', '습도_0', '최대온도_1', '평균온도_1', '습도_1', '최대온도_2', '최저온도_2', '평균온도_2', '습도_2', '일자별_domae가격_최대', '일자별_domae가격_평균', '일자별_somae가격_최대', '일자별_somae가격_평균', '일자별_somae가격_최소', '수출중량', '수출금액', '해당일자_전체평균가격']
2 품목의 selected_columns : ['단가', '거래량', '최저온도_0', '습도_0', '최저온도_1', '최저온도_2', '습도_2', '일자별_somae가격_평균', '일자별_somae가격_최소', '수출중량', '수출금액', '수입금액', '무역수지', '해당일자_전체평균가격']
3 품목의 selected_columns : ['단가', '거래량', '최저온도_1', '평균온도_1', '습도_2', '일자별_domae가격_최대', '일자별_domae가격_평균', '일자별_somae가격_평균', '일자별_somae가격_최소', '수출중량', '수출금액', '수입중량', '수입금액', '무역수지', '해당일자_전체평균가격']
4 품목의 selected_columns : ['단가', '거래량', '최저온도_0', '최저온도_2', '습도_2', '일자별_domae가격_평균', '일자별_domae가격_최소', '일자별_somae가격_최대', '일자별_somae가격_최소', '수출중량', '무역수지', '해당일자_전체평균가격']
5 품목의 selected_columns : ['단가', '거래량', '최대온도_0', '평균온도_0', '평균온도_1', '습도_1', '최대온도_2', '일자별_domae가격_평균', 

In [27]:
for i in range(37):
  print(globals()[f'total_{i}']['해당일자_전체평균가격'].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [28]:
def scaling_df(df):
  scaler = StandardScaler()
  
  
  tmp = df.drop('해당일자_전체평균가격',axis=1).copy()
  scaler.fit(tmp)
  df_scaled = scaler.transform(tmp)
  df_scaled = pd.DataFrame(data=df_scaled, columns = tmp.columns)
  df_scaled['해당일자_전체평균가격'] = df['해당일자_전체평균가격'].values 
  return df_scaled

for i in range(37):
  globals()[f'train_scaled_{i}'] = scaling_df(globals()[f'total_{i}'])

In [29]:
for i in range(37):
  print(globals()[f'train_scaled_{i}'].isna().sum().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [30]:
globals()[f'train_scaled_0']

,단가,습도_0,최저온도_1,평균온도_1,최대온도_2,습도_2,일자별_somae가격_최대,일자별_somae가격_최소,해당일자_전체평균가격
0,-1.562158,0.0,-0.839649,-1.055373,-0.883624,0.232086,-1.324653,-1.342761,6114.961048
1,6.636992,0.0,-0.606699,-0.673602,-0.789047,-0.017175,0.577939,0.777304,6114.961048
2,0.611502,0.0,-0.723174,-0.883576,-0.949829,-0.391066,0.577939,0.777304,6114.961048
3,1.017981,0.0,-0.785891,-0.845399,-0.770131,0.120809,0.577939,0.777304,6114.961048
4,1.516286,0.0,-0.839649,-0.902665,-0.940371,-0.773859,0.577939,0.777304,6114.961048
...,...,...,...,...,...,...,...,...,...
1456,-1.562158,0.0,-0.741093,-0.959930,-0.817420,-0.017175,-1.324653,-1.342761,7669.473042
1457,1.422662,0.0,-0.606699,-0.664058,-0.580976,0.427934,1.532417,0.856559,7334.439109
1458,1.433806,0.0,-0.785891,-1.084005,-1.309223,1.327053,1.532417,0.856559,7132.553300
1459,1.881196,0.0,-1.529540,-1.866634,-2.207711,-1.299087,1.532417,0.856559,6946.171088


In [31]:
try_cnt = 17 # 모델이나 체크포인트를 각자 다른 폴더에 저장해주기 위해
            # 새로운거 시도할때마다 숫자 바꿔서 내면된다.


In [32]:
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

In [33]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [34]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

In [35]:
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}')

    filename = f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

In [36]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

In [37]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

In [38]:
epoch = 1000
batch = 15

In [41]:
for i in range(0, 37):


    df_number = i
    df = globals()[f'train_scaled_{i}']
    
    # nan 처리
    df.fillna(0, inplace = True)
    
  

    

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격']], df['해당일자_전체평균가격']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch, batch)
    transformer.load_weights(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}') == False:
         os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}')

    
    
    
    # 모델 저장
    transformer.save(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1419/1419 [00:00<00:00, 22165.39it/s]


Epoch 1/1000
75/75 [============================>.] - ETA: 0s - loss: 9277248.0000
Epoch 1: val_loss improved from inf to 3344569.75000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-0-1000-15.h5
75/75 [==============================] - 6s 28ms/step - loss: 9209012.0000 - val_loss: 3344569.7500
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 1457059.1250
Epoch 2: val_loss improved from 3344569.75000 to 682554.12500, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-0-1000-15.h5
75/75 [==============================] - 2s 20ms/step - loss: 1436989.0000 - val_loss: 682554.1250
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 1481807.6250
Epoch 3: val_loss improved from 682554.12500 to 654900.87500, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-0-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 1460196.3750 - val_loss: 654900.8750
Epoch 4/1000
73/

100%|██████████| 1419/1419 [00:00<00:00, 21602.78it/s]


Epoch 1/1000
73/75 [===========================>..] - ETA: 0s - loss: 2135876.2500
Epoch 1: val_loss improved from inf to 279179.84375, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-1-1000-15.h5
75/75 [==============================] - 7s 24ms/step - loss: 2072668.7500 - val_loss: 279179.8438
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 590785.9375
Epoch 2: val_loss improved from 279179.84375 to 114276.08594, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-1-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 571845.6250 - val_loss: 114276.0859
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 586431.3750
Epoch 3: val_loss did not improve from 114276.08594
75/75 [==============================] - 1s 16ms/step - loss: 567191.2500 - val_loss: 158993.3750
Epoch 4/1000
73/75 [===========================>..] - ETA: 0s - loss: 605369.0625
Epoch 4: val_loss did not improve fro

100%|██████████| 1419/1419 [00:00<00:00, 23313.28it/s]


...

Epoch 1/1000
76/75 [==============================] - ETA: 0s - loss: 14109885.0000
Epoch 1: val_loss improved from inf to 3633237.75000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-5-1000-15.h5
75/75 [==============================] - 6s 25ms/step - loss: 14109885.0000 - val_loss: 3633237.7500
Epoch 2/1000
76/75 [==============================] - ETA: 0s - loss: 1844066.6250
Epoch 2: val_loss improved from 3633237.75000 to 1616361.37500, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-5-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 1844066.6250 - val_loss: 1616361.3750
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 1428398.0000
Epoch 3: val_loss improved from 1616361.37500 to 1436909.12500, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-5-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 1535201.1250 - val_loss: 1436909.1250
Epoch 4/1

100%|██████████| 1419/1419 [00:00<00:00, 22648.27it/s]


Epoch 1/1000
76/75 [==============================] - ETA: 0s - loss: 35812016.0000
Epoch 1: val_loss improved from inf to 44646400.00000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-6-1000-15.h5
75/75 [==============================] - 6s 25ms/step - loss: 35812016.0000 - val_loss: 44646400.0000
Epoch 2/1000
76/75 [==============================] - ETA: 0s - loss: 22720552.0000
Epoch 2: val_loss improved from 44646400.00000 to 28581850.00000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-6-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 22720552.0000 - val_loss: 28581850.0000
Epoch 3/1000
76/75 [==============================] - ETA: 0s - loss: 19829944.0000
Epoch 3: val_loss improved from 28581850.00000 to 26431098.00000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-6-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 19829944.0000 - val_loss: 26431098.00

100%|██████████| 1419/1419 [00:00<00:00, 22128.30it/s]


Epoch 1/1000
75/75 [============================>.] - ETA: 0s - loss: 3687531.7500
Epoch 1: val_loss improved from inf to 4728523.00000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-7-1000-15.h5
75/75 [==============================] - 6s 25ms/step - loss: 3704727.7500 - val_loss: 4728523.0000
Epoch 2/1000
76/75 [==============================] - ETA: 0s - loss: 1371253.1250
Epoch 2: val_loss improved from 4728523.00000 to 2382806.50000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-7-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 1371253.1250 - val_loss: 2382806.5000
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 1131899.6250
Epoch 3: val_loss improved from 2382806.50000 to 2326732.50000, saving model to /content/drive/MyDrive/농산물예측/aT_data/check17/transformer-7-1000-15.h5
75/75 [==============================] - 1s 18ms/step - loss: 1099912.7500 - val_loss: 2326732.5000
Epoch 4/100

In [42]:
for i in tqdm(range(10)):
  
  make_csv(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{i}/','test') 
  
  for j in range(37):
    # print(f'sep:{i}, 품목:{j}')
    # print(len(globals()[f'test_total_sep_{i}_{j}'])) # df row수 - 0이 있나 확인하기 위해서

    

    # 정규 표현식 적용
    globals()[f'test_total_sep_{i}_{j}'].columns = globals()[f'test_total_sep_{i}_{j}'].columns.str.replace(r'\([^)]*\)','',regex= True)

    # 스케일링 & 타겟값 변환
    #globals()[f'train_scaled_{i}'] = scaling_df(globals()[f'total_{i}'])
    globals()[f'test_total_sep_{i}_{j}'] = scaling_df(globals()[f'test_total_sep_{i}_{j}'])

    # train 데이터랑 똑같이 selection
    globals()[f'test_total_sep_{i}_{j}'] = globals()[f'test_total_sep_{i}_{j}'][list(globals()[f'train_scaled_{j}'].columns)]

    

    # 해당일자평균가격 테스트 데이터에서는 타겟값 없애고 돌리네
    globals()[f'test_total_sep_{i}_{j}'].drop('해당일자_전체평균가격',axis=1, inplace=True)


    file_number = j

    # nan 처리
    globals()[f'test_total_sep_{i}_{j}'].fillna(0, inplace = True)
    # 형상 맞추기 코드는 아직 넣지 않았다. (안넣어도 되지 않을까?)

    # x_test 생성
    df_test = astype_data(globals()[f'test_total_sep_{i}_{j}'].values.reshape(1, globals()[f'test_total_sep_{i}_{j}'].values.shape[0], globals()[f'test_total_sep_{i}_{j}'].values.shape[1]))



    # model test
    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}')

    # 해당하는 모델 불러오기
    model_test = tf.keras.models.load_model(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{file_number}-{epoch}-{batch}.h5')
    pred = model_test.predict(df_test)


    # 여기서 다시 형변환 해주자!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!np.expm1
    # pred = np.expm1(pred)

    

    # 결과 저장
    save_df = pd.DataFrame(pred).T
    save_df.to_csv(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}/predict_{file_number}.csv', index=False)


100%|██████████| 37/37 [00:00<00:00, 189.47it/s]

100%|██████████| 37/37 [00:00<00:00, 79.86it/s]

100%|██████████| 37/37 [00:02<00:00, 15.58it/s]

100%|██████████| 37/37 [00:02<00:00, 15.33it/s]

100%|██████████| 37/37 [00:11<00:00,  3.15it/s]

100%|██████████| 37/37 [00:10<00:00,  3.54it/s]

100%|██████████| 37/37 [00:03<00:00, 10.67it/s]

100%|██████████| 37/37 [00:02<00:00, 14.35it/s]

100%|██████████| 37/37 [00:11<00:00,  3.27it/s]

100%|██████████| 37/37 [00:11<00:00,  3.09it/s]

100%|██████████| 37/37 [00:03<00:00, 10.52it/s]

100%|██████████| 37/37 [00:02<00:00, 15.21it/s]

100%|██████████| 37/37 [00:11<00:00,  3.17it/s]

100%|██████████| 37/37 [00:11<00:00,  3.21it/s]

100%|██████████| 37/37 [00:03<00:00, 10.21it/s]

100%|██████████| 37/37 [00:02<00:00, 15.07it/s]

100%|██████████| 37/37 [00:10<00:00,  3.51it/s]

100%|██████████| 37/37 [00:11<00:00,  3.10it/s]

100%|██████████| 37/37 [00:03<00:00, 10.45it/s]

100%|██████████| 37/37 [00:02<00:00, 14.96it/s]

100%|██████████| 37

In [43]:
# 이부분 세부적으로 다 바꿔야한다.



for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:04<00:00,  2.07it/s]


In [44]:
pd.options.display.max_columns=37
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,품목10,품목11,품목12,품목13,품목14,품목15,품목16,품목17,품목18,품목19,품목20,품목21,품목22,품목23,품목24,품목25,품목26,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280,3361.030923,4911.899864,1173.018633,1337.03482,2053.354945,3003.205509,3148.414256,6573.839289,1089.126835,3838.815254,2254.390206,12538.931552,2990.863631,2965.711352,505.929304,937.367871,4147.240906,1231.11322,3323.910031,3056.787271,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,4363.2773,1025.380400,2056.491700,2243.165500,2459.392800,4494.140600,10272.226,2225.790000,4607.241700,1588.164600,1583.15100,4468.022500,3066.208300,2676.414800,7901.071300,1576.327600,2571.321300,2295.513200,6948.220000,2065.235000,1676.735100,528.880200,395.028200,3547.502200,1106.39930,1690.261200,2132.794000,4735.611000,484.014860,1086.239100,841.472900,3283.650100,3777.129200,7351.846000,3299.310800,2332.078100,1810.412500
2,4376.0054,1009.790470,2052.715800,2245.457000,2461.756600,4498.826000,10100.294,2235.514600,4605.363300,1588.122800,1583.08130,4458.986000,3065.659000,2678.416500,7893.069000,1581.891800,2585.134800,2294.456300,6941.982400,2066.582300,1681.460100,524.518800,411.006500,3562.684000,1108.73290,1614.690900,2138.867200,4748.017600,488.266050,1086.749900,843.383700,3296.601300,3766.500700,7362.293000,3319.573000,2335.734900,1814.273000
3,4368.1410,1020.591000,2050.284000,2246.834500,2472.237500,4502.787000,9822.758,2246.782700,4626.419400,1588.423300,1582.73690,4468.094700,3069.384000,2691.541000,7890.295000,1576.137300,2591.003700,2298.130000,6914.483400,2064.534000,1682.278100,532.657040,424.770540,3583.898400,1108.45630,1666.836300,2138.366200,4758.663600,483.864500,1083.538300,841.354860,3308.074700,3754.078600,7366.339000,3314.697800,2337.918200,1813.115100
4,4372.6950,1021.180100,2049.494100,2249.164000,2470.855200,4504.714000,9736.144,2250.494000,4603.014600,1588.471100,1582.18800,4456.674300,3066.288600,2701.479700,7902.788600,1582.182000,2598.765900,2304.747600,6904.107400,2066.848600,1684.719100,530.004330,412.511750,3592.571500,1106.33800,1738.143400,2144.398200,4768.712400,490.275180,1086.581900,840.575260,3329.013400,3772.426500,7360.354500,3321.473100,2341.173000,1815.968000
5,4376.0130,1012.240500,2046.172000,2248.615000,2470.610400,4510.641000,9848.188,2257.663300,4610.831500,1588.501600,1581.56570,4469.873000,3063.538800,2701.593500,7887.092000,1579.827800,2607.422600,2311.710700,6863.033000,2067.322000,1687.013800,530.390400,421.690120,3603.020000,1107.79380,1642.286500,2150.013400,4780.249500,483.289800,1082.476100,841.002600,3331.508300,3748.737500,7369.337000,3329.497300,2344.488800,1813.274000
6,4381.2560,1028.929100,2044.332200,2249.211200,2470.554400,4516.108000,9818.868,2262.525000,4606.632300,1588.471700,1581.45240,4464.326000,3061.246000,2709.084200,7885.292000,1579.035500,2619.433800,2309.453400,6858.658000,2069.447800,1687.211900,530.351700,403.211400,3612.137000,1100.70470,1637.391000,2144.477500,4792.556000,482.067000,1084.993200,839.123500,3341.713900,3719.226800,7368.860400,3331.819600,2346.578000,1818.231100
7,4368.9087,1003.084400,2042.408600,2250.895000,2471.116700,4518.214000,9844.656,2266.632300,4615.229500,1588.661300,1581.13730,4472.728500,3058.918200,2695.567600,7885.981000,1581.645400,2625.879600,2317.133300,6862.085400,2073.890400,1685.804400,537.248540,404.707860,3621.046600,1100.12290,1602.075300,2149.125700,4803.294000,482.849370,1087.250100,839.597530,3350.736300,3726.960200,7369.940000,3342.400100,2349.782500,1814.060000
8,4377.3010,1012.228600,2041.653100,2249.627400,2472.195300,4531.210000,9712.606,2269.921000,4601.515000,1588.513700,1580.94060,4462.956500,3056.403800,2722.192000,7896.580000,1580.104400,2633.073500,2318.480700,6785.822300,2075.335000,1690.135100,533.439150,416.411250,3631.982700,1102.21770,1548.632200,2143.011700,4811.292000,483.508500,

In [45]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

In [46]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [47]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('/content/drive/MyDrive/농산물예측/aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'/content/drive/MyDrive/농산물예측/submit{try_cnt}.csv',index=False)

In [48]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,품목10 변동률,품목11 변동률,품목12 변동률,품목13 변동률,품목14 변동률,품목15 변동률,...,품목19 변동률,품목20 변동률,품목21 변동률,품목22 변동률,품목23 변동률,품목24 변동률,품목25 변동률,품목26 변동률,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.127134,-0.247216,-0.293249,-0.340260,-0.377023,-0.514354,2.780334,-0.337766,-0.062025,0.353913,0.184076,1.175962,0.020979,-0.149917,0.201896,0.447332,...,-0.309485,-0.434626,0.045364,-0.578577,-0.144611,-0.101302,-0.491484,-0.302276,-0.451948,-0.195996,-0.017345,-0.462755,-0.096276,-0.307547,0.308347,-0.369231,-0.197247,-0.132809
1,0,d+2,0.130422,-0.258661,-0.294547,-0.339586,-0.376425,-0.513848,2.717060,-0.334872,-0.062407,0.353877,0.184024,1.171561,0.020796,-0.149281,0.200679,0.452440,...,-0.309035,-0.433033,0.036743,-0.561531,-0.140951,-0.099406,-0.514219,-0.300289,-0.450513,-0.188934,-0.016883,-0.461535,-0.092711,-0.309496,0.310206,-0.365358,-0.195988,-0.130960
2,0,d+3,0.128391,-0.250732,-0.295383,-0.339181,-0.373770,-0.513420,2.614923,-0.331520,-0.058120,0.354133,0.183766,1.175997,0.022036,-0.145112,0.200257,0.447157,...,-0.309720,-0.432757,0.052829,-0.546848,-0.135835,-0.099631,-0.498531,-0.300453,-0.449280,-0.196246,-0.019788,-0.462830,-0.089554,-0.311773,0.310926,-0.366290,-0.195237,-0.131515
3,0,d+4,0.129567,-0.250300,-0.295654,-0.338496,-0.374120,-0.513212,2.583048,-0.330416,-0.062885,0.354174,0.183356,1.170435,0.021005,-0.141955,0.202157,0.452707,...,-0.308946,-0.431934,0.047586,-0.559925,-0.133744,-0.101352,-0.477079,-0.298480,-0.448118,-0.185597,-0.017035,-0.463328,-0.083791,-0.308409,0.309861,-0.364994,-0.194116,-0.130148
4,0,d+5,0.130424,-0.256863,-0.296796,-0.338657,-0.374182,-0.512571,2.624282,-0.328282,-0.061294,0.354200,0.182890,1.176863,0.020090,-0.141919,0.199770,0.450545,...,-0.308788,-0.431160,0.048349,-0.550134,-0.131225,-0.100169,-0.505917,-0.296643,-0.446782,-0.197201,-0.020749,-0.463055,-0.083104,-0.312752,0.311459,-0.363460,-0.192975,-0.131439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.315563,-0.294538,0.517746,-0.027588,-0.137611,-0.024437,-0.292025,NaN,-0.163089,-0.263739,-0.316575,-0.353504,-0.214992,-0.338639,0.144461,-0.277591,...,0.389910,0.556477,-0.375792,-0.289497,0.792341,-0.008557,-0.366522,0.396114,-0.081817,-0.073207,-0.087892,-0.152119,0.025682,-0.169873,1.689788,0.750789,-0.034596,0.163012
146,9,d+12,-0.314320,-0.287023,0.515570,-0.028304,-0.137783,-0.025348,-0.287594,NaN,-0.164038,-0.263926,-0.316824,-0.353733,-0.215837,-0.342832,0.144421,-0.278624,...,0.390793,0.557164,-0.373180,-0.307038,0.790785,-0.000075,-0.417863,0.394084,-0.081589,-0.067754,-0.091781,-0.153674,0.028624,-0.170006,1.686174,0.744950,-0.034207,0.161094
147,9,d+13,-0.315636,-0.292982,0.513992,-0.028636,-0.141238,-0.024575,-0.274127,NaN,-0.163841,-0.264068,-0.316876,-0.353771,-0.215857,-0.346155,0.142965,-0.281058,...,0.394285,0.559368,-0.380756,-0.296589,0.794356,-0.006158,-0.373066,0.391550,-0.081599,-0.071780,-0.092443,-0.152914,0.028387,-0.175643,1.687887,0.753689,-0.033325,0.167779
148,9,d+14,-0.314352,-0.295481,0.511644,-0.029611,-0.140017,-0.024457,-0.253886,NaN,-0.163575,-0.264135,-0.316897,-0.354429,-0.215901,-0.348805,0.141517,-0.279738,...,0.393540,0.559183,-0.378326,-0.282483,0.790901,-0.008084,-0.380274,0.391104,-0.081238,-0.076592,-0.093147,-0.153986,0.028727,-0.176536,1.686111,0.744518,-0.032660,0.160881


In [49]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,품목10 변동률,품목11 변동률,품목12 변동률,품목13 변동률,품목14 변동률,품목15 변동률,...,품목19 변동률,품목20 변동률,품목21 변동률,품목22 변동률,품목23 변동률,품목24 변동률,품목25 변동률,품목26 변동률,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,품목10 변동률,품목11 변동률,품목12 변동률,품목13 변동률,품목14 변동률,품목15 변동률,...,품목19 변동률,품목20 변동률,품목21 변동률,품목22 변동률,품목23 변동률,품목24 변동률,품목25 변동률,품목26 변동률,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.127134,-0.247216,-0.293249,-0.340260,-0.377023,-0.514354,2.780334,-0.337766,-0.062025,0.353913,0.184076,1.175962,0.020979,-0.149917,0.201896,0.447332,...,-0.309485,-0.434626,0.045364,-0.578577,-0.144611,-0.101302,-0.491484,-0.302276,-0.451948,-0.195996,-0.017345,-0.462755,-0.096276,-0.307547,0.308347,-0.369231,-0.197247,-0.132809
1,0,d+2,0.130422,-0.258661,-0.294547,-0.339586,-0.376425,-0.513848,NaN,-0.334872,-0.062407,0.353877,0.184024,1.171561,0.020796,-0.149281,0.200679,0.452440,...,-0.309035,-0.433033,0.036743,-0.561531,-0.140951,-0.099406,-0.514219,-0.300289,-0.450513,-0.188934,-0.016883,-0.461535,-0.092711,-0.309496,0.310206,-0.365358,-0.195988,-0.130960
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,2.614923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.129567,-0.250300,-0.295654,-0.338496,-0.374120,-0.513212,NaN,-0.330416,-0.062885,0.354174,0.183356,1.170435,0.021005,-0.141955,0.202157,0.452707,...,-0.308946,-0.431934,0.047586,-0.559925,-0.133744,-0.101352,-0.477079,-0.298480,-0.448118,-0.185597,-0.017035,-0.463328,-0.083791,-0.308409,0.309861,-0.364994,-0.194116,-0.130148
4,0,d+5,0.130424,-0.256863,-0.296796,-0.338657,-0.374182,-0.512571,NaN,-0.328282,-0.061294,0.354200,0.182890,1.176863,0.020090,-0.141919,0.199770,0.450545,...,-0.308788,-0.431160,0.048349,-0.550134,-0.131225,-0.100169,-0.505917,-0.296643,-0.446782,-0.197201,-0.020749,-0.463055,-0.083104,-0.312752,0.311459,-0.363460,-0.192975,-0.131439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.315563,-0.294538,0.517746,-0.027588,-0.137611,-0.024437,-0.292025,NaN,-0.163089,-0.263739,-0.316575,-0.353504,-0.214992,-0.338639,0.144461,-0.277591,...,0.389910,0.556477,-0.375792,-0.289497,0.792341,-0.008557,-0.366522,0.396114,-0.081817,-0.073207,-0.087892,-0.152119,0.025682,-0.169873,1.689788,0.750789,-0.034596,0.163012
146,9,d+12,-0.314320,-0.287023,0.515570,-0.028304,-0.137783,-0.025348,-0.287594,NaN,-0.164038,-0.263926,-0.316824,-0.353733,-0.215837,-0.342832,0.144421,-0.278624,...,0.390793,0.557164,-0.373180,-0.307038,0.790785,-0.000075,-0.417863,0.394084,-0.081589,-0.067754,-0.091781,-0.153674,0.028624,-0.170006,1.686174,0.744950,-0.034207,0.161094
147,9,d+13,-0.315636,-0.292982,0.513992,-0.028636,-0.141238,-0.024575,-0.274127,NaN,-0.163841,-0.264068,-0.316876,-0.353771,-0.215857,-0.346155,0.142965,-0.281058,...,0.394285,0.559368,-0.380756,-0.296589,0.794356,-0.006158,-0.373066,0.391550,-0.081599,-0.071780,-0.092443,-0.152914,0.028387,-0.175643,1.687887,0.753689,-0.033325,0.167779
148,9,d+14,-0.314352,-0.295481,0.511644,-0.029611,-0.140017,-0.024457,-0.253886,NaN,-0.163575,-0.264135,-0.316897,-0.354429,-0.215901,-0.348805,0.141517,-0.279738,...,0.393540,0.559183,-0.378326,-0.282483,0.790901,-0.008084,-0.380274,0.391104,-0.081238,-0.076592,-0.093147,-0.153986,0.028727,-0.176536,1.686111,0.744518,-0.032660,0.160881
